In [2]:
import requests
res = requests.get('https://www.moea.gov.tw/MNS/populace/news/News.aspx?kind=1&menu_id=40&history=y')
#res.text

In [ ]:
from selenium import webdriver as wb
import re
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
import requests

options = wb.ChromeOptions()    #設定webdriver的選項
options.add_experimental_option('excludeSwitches', ["enable-automation"])  #在選項中加入" 開發者模式" 
browser=wb.Chrome(options=options)   #在webdriver中加入選項 
browser.get("https://www.moea.gov.tw/MNS/populace/news/News.aspx?kind=1&menu_id=40&history=y")  #中華民國經濟部網址


In [ ]:
#設定一個定義處理重複把網址變成soup
def make_soup(url_link):
    response = requests.get(url_link)
    soup = BeautifulSoup(response.text, 'lxml')
    return soup

In [ ]:
#抓取目標網站1000筆資料
content_data_lst = [ ]
title_lst = [ ] 
time_lst = [ ]
count = 0


while 1:
    #抓取當頁10個標題連結
    soup = BeautifulSoup(browser.page_source, 'lxml')
    #解析網頁
    link = soup.select('a[id^= holderContent_grdNews_lnkTitle] ')
    #發現網頁有規律故抓取後面並去掉 '' . ''
    domain = 'https://www.moea.gov.tw/MNS/populace'
        
    for ele in link:
        complete_link = domain + ele.get('href').strip(' . ')    
        source = make_soup(complete_link)       #運用前面連結 每抓取一次就把抓到的連結做一次湯

        source_ans = source.find_all('div',{'class':"div-left-info"})
        for i in source_ans:
            source_ans_s = i.text
            clear = source_ans_s.replace('\n','')
            source_clear = re.sub('\r','',clear)  

        content_data_lst.append(source_clear)     #使用 content_data_lst 存取每個標題下的內文

        title = source.find_all('div',{'class':"div-info-title"})
        for j in title:
            title_s = j.text
            clears = title_s.replace('\n','')
            title_clear = re.sub('\r','',clears)
        title_lst.append(title_clear)      #使用 title_lst 抓取 每篇標題

        times = source.find_all('div',{'class': "div-begin-date"})
        for k in times:
            times_s = k.text
            clearss = times_s.replace('\n','')
            times_clear = re.sub('\r','',clearss)

        time_lst.append(times_clear)  #使用time_lst 抓取 每篇發文時間

    #抓完10筆資料點擊下一頁
    submit_ele = browser.find_element_by_xpath('//*[@id="holderContent_grdNews_uctlPages_dltPage_btnNext"]')
    submit_ele.click()
    #設定抓取時間
    #time.sleep(random.randint(1,2))

    count += 1
    if count == 100:
        break


In [ ]:
ans = list(zip( title_lst, time_lst,content_data_lst,))

In [ ]:
df = pd.DataFrame(ans, columns = ['標題', '時間','內容'])
df